<a href="https://colab.research.google.com/github/AnupJoseph/adv-python/blob/master/Histogram_Plotting_%2B_Seaborn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from collections import Counter
import random
import numpy as np

In [3]:
values = (0, 1, 1, 1, 2, 3, 7, 7, 23)
value_count = Counter(values)
value_count

Counter({0: 1, 1: 3, 2: 1, 3: 1, 7: 2, 23: 1})

In [6]:
def ascii_histogram(seq) -> None:
  """
    A horizontal frequency-table/histogram plot.
  """
  counted = Counter(seq)
  for k in sorted(counted):
    print("{0:5d} {1}".format(k,'+'*counted[k]))

In [7]:
random.seed(1)

vals = [1, 3, 4, 6, 8, 9, 10]

# Each value will appear between 5 and 15 times
freq = (random.randint(5,15) for _ in vals)

# Creating
data = []
for f,v in zip(freq,vals):
  data.extend([v]*f)

ascii_histogram(data)

    1 +++++++
    3 ++++++++++++++
    4 ++++++
    6 +++++++++
    8 ++++++
    9 ++++++++++++
   10 ++++++++++++


In [ ]:
# Alright till now we have been cheating a bit and make it sound as if the about function is actually a histogram
# It isn't though and hence now we switch our focus to that

# More technically, it can be used to approximate the probability density function (PDF) of the underlying variable.If that sound awkward then thats fine.
# Moving on from the “frequency table” above, a true histogram first “bins” the range of values and then counts the number of values that fall into each bin

In [9]:
np.random.seed(444)
np.set_printoptions(precision=3)

d = np.random.laplace(loc=15,scale=3,size=500)
d[:5]

array([18.406, 18.087, 16.004, 16.221,  7.358])

In [10]:
# In the above case the laplacian distribution is a continous function and tracking down its last decimal place is kind of useless
# Instead, you can bin or “bucket” the data and count the observations that fall into each bin
# using the standard numpy histogram function

hist,bin_edges = np.histogram(d)

print(hist,bin_edges,sep='\n')

[ 13  23  91 261  80  21   7   2   1   1]
[ 2.11   5.874  9.638 13.402 17.166 20.93  24.694 28.458 32.222 35.986
 39.749]


In [11]:
print(hist.size,bin_edges.size,sep='\n')

10
11


In [16]:
first_edge,last_edge = min(values),max(values)

n_equal_bins = 10
bin_edges = np.linspace(start = first_edge,stop=last_edge,num=n_equal_bins+1,endpoint=True)

bin_edges

array([ 0. ,  2.3,  4.6,  6.9,  9.2, 11.5, 13.8, 16.1, 18.4, 20.7, 23. ])